In [1]:
datadir = "../data/"
datafilename = "Certainty.xlsx"
datafilepath = datadir + datafilename

In [2]:
from itertools import combinations
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


# Read in network information from sheet P0
# For now we only use the folloiwng columns in each sheet
cols_used = ['Link ID','ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','Link Length(miles)','# of lanes-A','Capacity-A (veh/h)',
            'auto volume(2010)-A','AADT(2010)-A','Speed(mph)-A','VMT-A']
# # For now we assume all occurrences of the same link in the same sheet share the same attribute values
df_p0= pd.read_excel(datafilepath, sheet_name='P0',usecols=cols_used).dropna(subset=['Link ID'])
# df_p0_unique = df_p0.drop_duplicates(subset=['Link ID']).dropna(subset=['Link ID', 'A X_COORD', 'A Y_COORD', 'B X_COORD', 'B Y_COORD'])


In [3]:
### Only Read in Project 3, 4 and 5
integers = [ 3, 4, 5]
def generate_combinations(integers):
    all_combinations = []
    # Loop through lengths from 1 to 6
    for length in range(1, 1+len(integers)):
        # Generate combinations of the current length
        comb = combinations(integers, length)
        # Convert each combination to a string, add "p", and add to the list
        all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
    return all_combinations
# Generate all combinations
sheet_names = generate_combinations(integers)

# Insert "P0" at the beginning of the list
sheet_names.insert(0, "P0")


In [4]:
##### Read in all 2^6 sheets of data ###


# ## Create a list of sheet names ##

# # List of the first 6 integers
# integers = [1, 2, 3, 4, 5, 6]

# # Function to generate combinations
# def generate_combinations(integers):
#     all_combinations = []
#     # Loop through lengths from 1 to 6
#     for length in range(1, 7):
#         # Generate combinations of the current length
#         comb = combinations(integers, length)
#         # Convert each combination to a string, add "p", and add to the list
#         all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
#     return all_combinations

# # Generate all combinations
# sheet_names = generate_combinations(integers)

# # Insert "P0" at the beginning of the list
# sheet_names.insert(0, "P0")

In [5]:
# # Preparing the sample set
# dataset = []
# for ii in range(len(sheet_names)):
#     # read the correct sheet 
#     df= pd.read_excel(datafilepath, sheet_name=sheet_names[ii],usecols=cols_used).dropna(subset=['Link ID'])
    
#     # dataframe for edge attributes
#     df_edge_attr= np.column_stack((df['Link Length(miles)'],df['# of lanes-A'],df['Capacity-A (veh/h)'], df['Speed(mph)-A'],df['auto volume(2010)-A']))
#     # edge_attr = torch.tensor(df_edge_attr, dtype=torch.float)

#     df_edge_labels = df['AADT(2010)-A'].to_numpy()
#     # edge_labels = torch.tensor(df['AADT(2010)-A'].to_numpy(),dtype=torch.float)
#     # new_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=edge_labels)
    
#     # sample = (df_edge_attr, df_edge_labels)
#     dataset.append(sample)

# X = [sample[0] for sample in dataset]
# y = [sample[1] for sample in dataset]


In [6]:
# normalized data
# Preparing the sample set
dataset = np.zeros((len(sheet_names)*650,6))
for ii in range(len(sheet_names)):
    # read the correct sheet 
    df= pd.read_excel(datafilepath, sheet_name=sheet_names[ii],usecols=cols_used).dropna(subset=['Link ID'])
    
    # dataframe for edge attributes
    df_edge_attr= np.column_stack((df['Link Length(miles)'],df['# of lanes-A'],df['Capacity-A (veh/h)'], df['Speed(mph)-A'],df['auto volume(2010)-A']))
 
    df_edge_labels = df['AADT(2010)-A'].to_numpy()
  
    sample = np.column_stack((df_edge_attr, df_edge_labels))
    dataset[ii*650:(ii+1)*650]= sample


df_dataset = pd.DataFrame(dataset, columns=['Link Length(miles)','# of lanes-A','Capacity-A (veh/h)','Speed(mph)-A','auto volume(2010)-A','AADT(2010)-A'])

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_dataset = scaler.fit_transform(df_dataset)

X = scaled_dataset[:,:5]
y = scaled_dataset[:,-1]

In [7]:
print(X)

[[0.03195832 0.14285714 0.19816724 0.51226189 0.05863944]
 [0.07325476 0.14285714 0.19816724 0.51226189 0.0962912 ]
 [0.04867793 0.14285714 0.19816724 0.5118514  0.10709455]
 ...
 [0.02856299 0.14285714 0.05211913 0.22260648 0.0511708 ]
 [0.00846723 0.         0.05211913 0.21372917 0.05832164]
 [0.02118534 0.14285714 0.05211913 0.23220691 0.0371458 ]]


In [8]:


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)




# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train,dtype=torch.float)
X_val = torch.tensor(X_val,dtype=torch.float)
y_val = torch.tensor(y_val,dtype=torch.float)


In [10]:
 
# Define the neural network model
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = RegressionModel()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 50
batch_size = 1

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for i in range(X_train.shape[0]):
        optimizer.zero_grad()
        outputs = model(X_train[i])
        loss = criterion(outputs.squeeze(), y_train[i])
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {format(epoch_loss/X_train.shape[0],".2e")}')

# Evaluate the model
model.eval()
val_loss = 0.0
relative_val_loss = 0.0
sum_edge_labels = 0.0
with torch.no_grad():
    for i in range(X_val.shape[0]):
        outputs = model(X_val[i])
        loss = criterion(outputs.squeeze(), y_val[i])
        val_loss += loss.item()
        
        # Calculate Relative Validation Loss
        edge_labels_tensor = criterion(torch.zeros(torch.Tensor.size(y_val[i])), y_val[i])
        sum_edge_labels += edge_labels_tensor.item()
        relative_loss = loss/edge_labels_tensor
        relative_val_loss += relative_loss.item()
        
abs_val_loss = val_loss/X_val.shape[0]
print(f'Absolute Validation Loss: {format(abs_val_loss,".2e")}')

# calculate average edge_labels_tensor.item()
ave_edge_labels = sum_edge_labels/X_val.shape[0]
print("\n Relative Loss = ", format(abs_val_loss, '.2e'), " / ", format(ave_edge_labels,'.2e'), " = ", format(abs_val_loss/ave_edge_labels,'.2e'), "\n")
 

# Make predictions
model.eval()
with torch.no_grad():
    for i in range(int(X_val.shape[0]/650)):
        pred_scaled = model(X_val[:(i+1)*650,:]).squeeze()
        # transform the prediction into unscaled values

        pred_scaled = pred_scaled.numpy() # convert predicted label to np array
        data_pred_scaled = np.hstack((X_val[:(i+1)*650,:], pred_scaled.reshape(-1,1)))
     
        unscaled_pred = scaler.inverse_transform(data_pred_scaled)

        print(f'Sample {i+1} Predictions:\n', unscaled_pred[:,-1])

Epoch 1/50, Loss: 1.68e-03
Epoch 2/50, Loss: 9.63e-04
Epoch 3/50, Loss: 8.70e-04
Epoch 4/50, Loss: 8.02e-04
Epoch 5/50, Loss: 7.48e-04
Epoch 6/50, Loss: 7.10e-04
Epoch 7/50, Loss: 6.81e-04
Epoch 8/50, Loss: 6.53e-04
Epoch 9/50, Loss: 6.28e-04
Epoch 10/50, Loss: 6.08e-04
Epoch 11/50, Loss: 5.87e-04
Epoch 12/50, Loss: 5.68e-04
Epoch 13/50, Loss: 5.52e-04
Epoch 14/50, Loss: 5.38e-04
Epoch 15/50, Loss: 5.27e-04
Epoch 16/50, Loss: 5.13e-04
Epoch 17/50, Loss: 5.15e-04
Epoch 18/50, Loss: 5.05e-04
Epoch 19/50, Loss: 4.95e-04
Epoch 20/50, Loss: 4.85e-04
Epoch 21/50, Loss: 4.84e-04
Epoch 22/50, Loss: 4.74e-04
Epoch 23/50, Loss: 4.68e-04
Epoch 24/50, Loss: 4.58e-04
Epoch 25/50, Loss: 4.59e-04
Epoch 26/50, Loss: 4.45e-04
Epoch 27/50, Loss: 4.43e-04
Epoch 28/50, Loss: 4.41e-04
Epoch 29/50, Loss: 4.33e-04
Epoch 30/50, Loss: 4.25e-04
Epoch 31/50, Loss: 4.24e-04
Epoch 32/50, Loss: 4.18e-04
Epoch 33/50, Loss: 4.13e-04
Epoch 34/50, Loss: 4.19e-04
Epoch 35/50, Loss: 4.13e-04
Epoch 36/50, Loss: 4.03e-04
E